Problem statement

In this dataset, you are presented pairs of phrases (an anchor and a target phrase) and asked to rate how similar they are on a scale from 0 (not at all similar) to 1 (identical in meaning). 

Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Load datasets

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Read datasets

In [ ]:
train = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv")
test = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv")
submission = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/sample_submission.csv")

In [ ]:
train

In [ ]:
test

In [ ]:
submission

Analyse score

In [ ]:
sns.distplot(train.score)

In [ ]:
plt.boxplot(train.score)

Define target

In [ ]:
target = train.score

Combine train and test

In [ ]:
combi = train.drop(['score'], axis=1).append(test)
combi

Match phrase similarities

In [ ]:
!pip install -U spacy

In [ ]:
!python -m spacy download en

In [ ]:
import spacy
nlp = spacy.load("en_core_web_lg")

simularity = []

for i in range(len(combi)):
    anchor = nlp(combi['anchor'].values[i])
    text_target = nlp(combi['target'].values[i])
    sim = anchor.similarity(text_target)
    simularity.append(sim)
    
print(len(simularity))

Create new column in combi

In [ ]:
combi['similar'] = simularity
combi

Ordinal encode context

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder()

combi['context'] = enc.fit_transform(combi['context'].values.reshape(-1,1))
combi

Define X and y

In [ ]:
features = ['similar', 'context']

y = target
X = combi[features][: len(train)]
X_test = combi[features][len(train) :]

Define model

In [ ]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor(random_state=42).fit(X, y)
print(model.score(X, y))

Predict on test set

In [ ]:
prediction = model.predict(X_test)
prediction

Prepare submission

In [ ]:
submission['score'] = prediction
submission.to_csv("submission.csv", index=False)
submission